In [7]:
#Importing all needed packages
using PyPlot
using Suppressor
using Formatting
import Pkg
using Base

#Defining custom macros
#Making custom macro to get variable name as string for plots
macro Name(arg)
    string(arg)
end

@Name (macro with 1 method)

In [8]:
#= notebook I am using to calculate the momentum flux τ
of an idealized ice floe with properties taken from the literature.
I will calculate the momentum flux as a function of the form drag for
sails and keels, floe edges, and the atmospheric and oceanic skin drag.
=#

#Defining constants at the start
#Floe size(m) (Just looking at one floe so don't need this...)
L = 10
#Freeboard(m)
H_f = 1
#Draft(m) 
D = 1
#Pond size(m). Note there are no ponds in Antarctica so set this to 0.
L_p = 0
#Ratio of keel depth and sail height (Worby (2008))
R_h = 4.4
#Ratio of average distance between keels and average distance between sails
R_d = 1
#weight variables of sails and keels (how many sails vs how many keels)
#For now assuming there is an equal amount of each
#α in Tsamados.. here naming W_s for weighting of sails
W_s = 0.5
#β in Tsamados... here naming W_k for weighting of keels
W_k = 0.5
#Slope of sails(rad) (Worby (2008))
α_r = 0.45
#Slope of keels(rad) (Worby (2008))
α_k = 0.45
#Attenuation parameter in sheltering function (given by Tsamados)
s_l = 0.18
#roughness length of level ice (given by CICE)
z_oi = 5e-4
#Ice concentration(fraction of 1.0)
A = 0.5
#Sail height(m) (Worby (2008))
#H_s = 0.57
#ratio of aice to ardg, Ridged ice area fraction
R_f = 1/0.11
#Coefficents c_ra and c_kw
c_ra = 0.2
c_kw = 0.2

#Now defining geometric properties of the sails and keels that depend on the initial definition of H_s
#H_s -> D_s & H_k & X_s
#D_s -> D_k
#H_k -> X_k

#Distance between sails(m) (Taken from equation in Tsamados). aice/ardg = 0.11 as given in Worby(2008)
D_s(H_s) = 2*H_s*R_f*(W_s/tan(α_r) + (W_k/tan(α_k))*(R_h/R_d))
#Keel height(m)
H_k(H_s) = R_h*H_s
#Distance between keels(m)
D_k(D_s) = R_d*D_s
#Base width of sails(m)
X_s(H_s) = (2*H_s)/(tan(α_r))
#Base width of keels(m)
X_k(H_k) = (2*H_k)/(tan(α_k))

#now defining individual drag components
#first sheltering function
S_c(D,H) = (1-exp((-s_l*D)/H))^(0.5)
#Now the form drag coefficent from sails
C_dar(H_s,D_s) = 0.5*c_ra*S_c(D_s,H_s)^(2.0)*(H_s/D_s)*A*(log(H_s/z_oi)/log(10/z_oi))^(2.0)
#Now the form drag coefficent from keels
C_dwr(H_k,D_k) = 0.5*c_kw*S_c(D_k,H_k)^(2.0)*(H_k/D_k)*A*(log(H_k/z_oi)/log(10/z_oi))^(2.0)

C_dwr (generic function with 1 method)

In [19]:
#Now we want to vary H_s, see what happens to drag
function plotdrag(H_s)
    D_s_temp = [D_s(i) for i in H_s]
    H_k_temp = [H_k(i) for i in H_s]
    D_k_temp = [D_k(j) for j in D_s_temp]
    # zip stores two arrays ["a","b"] and [1,2] as [["a",1],["b",2]]
    S_zip = zip(H_s,D_s_temp)
    K_zip = zip(H_k_temp,D_k_temp)
    totalsail = [C_dar(arr[1],arr[2]) for arr in S_zip]
    totalkeel = [C_dwr(arr[1],arr[2]) for arr in K_zip]
    totaldrag = totalsail + totalkeel
    println(totaldrag)
    #plot!(H_s,D_s_temp,totalkeel,st=:surface,label="Keels")
    #plot!(H_s,D_s_temp,totaldrag,st=:surface,label="Total")
end

plotdrag (generic function with 1 method)

In [20]:
H_s_range = range(0.2,stop=1.0,length=50)

plotdrag(H_s_range)
savefig("testplot.png")
println("plot working")

[0.00139344, 0.0014238, 0.00145225, 0.00147902, 0.00150432, 0.0015283, 0.0015511, 0.00157284, 0.00159363, 0.00161353, 0.00163264, 0.00165102, 0.00166871, 0.00168578, 0.00170227, 0.00171822, 0.00173367, 0.00174864, 0.00176318, 0.00177729, 0.00179102, 0.00180438, 0.00181739, 0.00183007, 0.00184243, 0.0018545, 0.00186629, 0.00187782, 0.00188908, 0.0019001, 0.00191089, 0.00192146, 0.00193181, 0.00194196, 0.00195191, 0.00196168, 0.00197126, 0.00198067, 0.00198991, 0.00199899, 0.00200792, 0.0020167, 0.00202533, 0.00203382, 0.00204218, 0.0020504, 0.0020585, 0.00206648, 0.00207434, 0.00208209]
plot working


In [169]:
x = range(0.2,stop=1.0,length=50)
y = range(0.5,stop=80,length=50)
z = [i^(2.0) for i in x]
surf(x,y,z)

PyCall.PyError: PyError ($(Expr(:escape, :(ccall(#= /home/ben/.julia/packages/PyCall/RQjD7/src/pyfncall.jl:44 =# @pysym(:PyObject_Call), PyPtr, (PyPtr, PyPtr, PyPtr), o, pyargsptr, kw))))) <class 'RuntimeError'>
RuntimeError('Error in qhull Delaunay triangulation calculation: singular input data (exitcode=2); use python verbose option (-v) to see original qhull error.')
  File "/home/ben/.julia/conda/3/lib/python3.7/site-packages/mpl_toolkits/mplot3d/axes3d.py", line 1945, in plot_trisurf
    tri, args, kwargs = Triangulation.get_from_args_and_kwargs(*args, **kwargs)
  File "/home/ben/.julia/conda/3/lib/python3.7/site-packages/matplotlib/tri/triangulation.py", line 167, in get_from_args_and_kwargs
    triangulation = Triangulation(x, y, triangles, mask)
  File "/home/ben/.julia/conda/3/lib/python3.7/site-packages/matplotlib/tri/triangulation.py", line 55, in __init__
    self.triangles, self._neighbors = _qhull.delaunay(x, y)
